<a href="https://colab.research.google.com/github/tnfru/colab_research/blob/master/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR100
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torchsummary import summary
import time

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', DEVICE)
torch.backends.cudnn.benchmark = True

using device: cuda


In [5]:
BATCH_SIZE = 128
IMG_SIZE = [32, 32]
TRAIN_MEANS = np.array([0.50707516, 0.48654887, 0.44091784])
TRAIN_STDS = np.array([0.26733429, 0.25643846, 0.27615047])
TEST_MEANS = np.array([0.50879641, 0.48739301, 0.44194221])
TEST_STDS = np.array([0.26825157, 0.25736374, 0.27709577])

In [6]:
tr_train = T.Compose([T.ToTensor(),
                      T.RandomHorizontalFlip(p=0.5),
                      T.Normalize(TRAIN_MEANS, TRAIN_STDS)])

tr_test = T.Compose([T.ToTensor(),
                     T.Normalize(TEST_MEANS, TEST_STDS)])

train = CIFAR100("./data", train=True, download=True, transform=tr_train)
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

test = CIFAR100("./data", train=False, download=True, transform=tr_test)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, pin_memory=True)

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
class IdentityModule(nn.Module):
  def __init__(self, num_channel):
    super(IdentityModule, self).__init__()

    self.hidden1 = nn.Sequential(
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU(),
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU()
    )

    self.hidden2 = nn.Sequential(
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU(),
        nn.Conv2d(num_channel, num_channel, 3, padding=1),
        nn.BatchNorm2d(num_channel),
        nn.ReLU()
    )
  
  def forward(self, x): # TODO swap Relu after add
    y1 = self.hidden1(x)
    y2 = self.hidden2(x)

    return x + y1 + y2

In [8]:
class ConvModule(nn.Module):
  def __init__(self, in_channel, out_channel, stride=2):
    super(ConvModule, self).__init__()

    self.hidden1 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 3, padding=1, stride=stride),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(),
        nn.Conv2d(out_channel, out_channel, 3, padding=1),
        nn.BatchNorm2d(out_channel),
        nn.ReLU()
    )
    self.hidden2 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 3, padding=1, stride=stride),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(),
        nn.Conv2d(out_channel, out_channel, 3, padding=1),
        nn.BatchNorm2d(out_channel),
        nn.ReLU()
    )
    self.hidden3 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 1, stride=stride),
        nn.BatchNorm2d(out_channel)
    )

  def forward(self, x):
    y1 = self.hidden1(x)
    y2 = self.hidden2(x)
    y3 = self.hidden3(x)

    return self.hidden1(x) + self.hidden2(x) + self.hidden3(x)


In [9]:
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    self.inp = nn.Sequential(
        nn.Conv2d(3, 16, 3, padding=1),
        nn.Dropout(0.1)
    )

    self.hidden1 = nn.Sequential(
        ConvModule(16, 64, stride=1),
        IdentityModule(64),
        nn.Dropout(0.15)
    )

    self.hidden2 = nn.Sequential(
        ConvModule(64, 128),
        IdentityModule(128),
        nn.Dropout(0.2)
    )

    self.hidden3 = nn.Sequential(
        ConvModule(128, 256),
        IdentityModule(256),
        nn.Dropout(0.25)
    )

    self.out = nn.Sequential(
        nn.AvgPool2d(kernel_size=8),
        nn.Flatten(),
        nn.Linear(256, 100),
        nn.Softmax(dim=1)
    )

  def forward(self, x):
    x = self.inp(x)
    x = self.hidden1(x)
    x = self.hidden2(x)
    x = self.hidden3(x)
    x = self.out(x)

    return x

In [27]:
def test_accuracy(model, history):
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      total_loss =0

      for data, target in test_loader:
          images = data.to(DEVICE)
          labels = target.to(DEVICE)
          outputs = model(images)

          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          accuracy = 100 * correct / total
          history.append(accuracy)

  model.train()

  return history

In [41]:
def train_model(model, opt, scheduler, num_epochs, history=[], verbose=True):
  criterion = nn.CrossEntropyLoss()

  for epoch in range(num_epochs):
    start_time = time.time()
    for i, (images, labels) in enumerate(train_loader):
      # Write tensor to GPU if available
      images, labels = images.to(DEVICE), labels.to(DEVICE)

      # Clear Gradient
      opt.zero_grad()

      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)

      # Backward pass
      loss.backward()
      opt.step()

    history = test_accuracy(model, history)

    if verbose:
      print('EPOCH: {} LR: {} Took Time: {:.4} Accuracy: {} %'.format(epoch, 
            scheduler.get_last_lr(), time.time() - start_time, history[-1]))
    
    scheduler.step()

In [42]:
model = ResNet()
model.to(DEVICE)
opt = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.MultiStepLR(opt, milestones=[25,70,100], gamma=0.1)
history = []

In [43]:
train_model(model, opt, scheduler, num_epochs=50, history=history)

EPOCH: 0 LR: [0.001] Took Time: 76.4 Accuracy: 4.22 %
EPOCH: 1 LR: [0.001] Took Time: 75.75 Accuracy: 5.17 %
EPOCH: 2 LR: [0.001] Took Time: 75.75 Accuracy: 5.42 %
EPOCH: 3 LR: [0.001] Took Time: 75.9 Accuracy: 5.98 %
EPOCH: 4 LR: [0.001] Took Time: 75.83 Accuracy: 6.45 %
EPOCH: 5 LR: [0.001] Took Time: 75.76 Accuracy: 7.41 %
EPOCH: 6 LR: [0.001] Took Time: 75.77 Accuracy: 8.35 %
EPOCH: 7 LR: [0.001] Took Time: 75.78 Accuracy: 8.83 %
EPOCH: 8 LR: [0.001] Took Time: 75.86 Accuracy: 8.85 %
EPOCH: 9 LR: [0.001] Took Time: 75.75 Accuracy: 9.55 %
EPOCH: 10 LR: [0.001] Took Time: 75.81 Accuracy: 10.24 %


KeyboardInterrupt: ignored